In [197]:
import pandas as pd
import numpy as np
import re
from difflib import Differ 
import difflib

In [198]:
# opening files

with open('Залог_старая_редакция_ред_от_05_05_2014.txt', 'r') as file1:
    data1 = file1.read()
    
with open('Залог_текущая_редакция_ред_от_11_03_2024.txt', 'r') as file2:
    data2 = file2.read()

In [200]:
def split_text_into_fragments_with_headers(text):
    # create pattern, find matches
    pattern = r'(Статья\s+\d+\.|Раздел\s+\d+\.|Подраздел\s+\d+\.|Глава\s+\d+\.)\s*(.+?)(?=(Статья\s+\d+\.|Раздел\s+\d+\.|Подраздел\s+\d+\.|Глава\s+\d+\.)|$)'
    fragments = re.findall(pattern, text, re.DOTALL)
    
    # storing fragments
    result = []
    for fragment in fragments:
        # join headings and fragments
        full_fragment = fragment[0].strip() + ' ' + fragment[1].strip()
        result.append(full_fragment.strip())

    return result

In [201]:
# head of article extraction | collection of heads | create DataFrame

def collect_heads_pd(articles):
    head_articles = []

    for t in articles:
        head = re.search(r'(Статья\s+\d+\.(\s+\d+\.)?)', t).group()
        head_articles.append(head)
    
    df = pd.DataFrame(articles, \
             head_articles).reset_index().rename(columns={'index': 'Номер статьи', 0: 'Текст статьи'})

    return df

In [202]:
# break text into articles | cutting headings

articles_old = split_text_into_fragments_with_headers(data1)[2:]
articles_new = split_text_into_fragments_with_headers(data2)[2:]

In [203]:
df_old = collect_heads_pd(articles_old).rename(columns={'Текст статьи': 'Старая редакция'})
df_new = collect_heads_pd(articles_new).rename(columns={'Текст статьи': 'Новая редакция'})

In [204]:
# merging df | filling NaN values

df_merged = df_new.merge(df_old, how='outer').fillna('none')

In [205]:
# Comparison | adding ratio

df_merged['Степень сходства'] = df_merged['Номер статьи']

for i in range(len(df_merged)):
    df_merged['Степень сходства'].iloc[i] = difflib.SequenceMatcher(None, df_merged['Новая редакция'].iloc[i].split(), \
                        df_merged['Старая редакция'].iloc[i].split()).ratio()

In [206]:
df_merged

,Номер статьи,Новая редакция,Старая редакция,Степень сходства
0,Статья 334.,Статья 334. Понятие залога\n \n 1. В с...,Статья 334. Понятие и основания возникновения ...,0.284314
1,Статья 334. 1.,Статья 334. 1. Основания возникновения залога\...,none,0.0
2,Статья 335.,Статья 335. Залогодатель\n \n 1. Залог...,Статья 335. Залогодатель\n\n 1. Залогодате...,0.11236
3,Статья 335. 1.,Статья 335. 1. Созалогодержатели\n \n ...,none,0.0
4,Статья 336.,Статья 336. Предмет залога\n \n 1. Пре...,Статья 336. Предмет залога\n\n 1. Предмето...,0.448133
5,Статья 337.,Статья 337. Обеспечиваемое залогом требование\...,Статья 337. Обеспечиваемое залогом требование\...,0.78
6,Статья 338.,Статья 338. Владение предметом залога\n \n...,Статья 338. Залог без передачи и с передачей з...,0.690476
7,Статья 339.,Статья 339. Условия и форма договора залога\n ...,"Статья 339. Договор о залоге, его форма и реги...",0.287059
8,Статья 339. 1.,Статья 339. 1. Государственная регистрация и у...,none,0.0
9,Статья 340.,Статья 340. Стоимость предмета залога\n \n...,"Статья 340. Имущество, на которое распространя...",0.102564


In [207]:
df_merged.to_csv("articles_pledge.csv")